### Importing neccessary libraries etc.

In [ ]:
import pandas as pd
import re
import numpy as np
#from ccdc.io import EntryReader
#from ccdc.io import CrystalReader
#csd_entry_reader = EntryReader('CSD')
#csd_crystal_reader = CrystalReader('CSD')

#### Continuing to properly parse the molecular formula.
(The code below is where I left off in the previous notebook.)

In [ ]:
store = []
i = 0
for entry in csd_entry_reader:
    store.append(((((entry.identifier, entry.formula, entry.crystal.cell_volume, entry.crystal.z_value, entry.crystal.z_prime)))))
    #unitform = df['Molecular Formula'].multiply(df['Z'])
    #store.append(unitform) <<< use after parsing formula
    i += 1
    if i == 10:
        break
headers = ["Refcode", "Molecular Formula", "Cell volume", "Z", "Z prime"]
df = pd.DataFrame(data = store, columns = headers)
df = df[df['Z'] > 0]
#print df
#df.to_csv("test2.csv")

allowedElements = {"H":0,"He":0,"Li":0,"Be":0,"B":0,"C":0,"N":0,"O":0,"F":0,"Ne":0,
"Na":0,"Mg":0,"Al":0,"Si":0,"P":0,"S":0,"Cl":0,"Ar":0,"K":0,"Ca":0,
"Sc":0,"Ti":0,"V":0,"Cr":0,"Mn":0,"Fe":0,"Co":0,"Ni":0,"Cu":0,"Zn":0,
"Ga":0,"Ge":0,"As":0,"Se":0,"Br":0,"Kr":0,"Rb":0,"Sr":0,"Y":0,"Zr":0,
"Nb":0,"Mo":0,"Tc":0,"Ru":0,"Rh":0,"Pd":0,"Ag":0,"Cd":0,"In":0,"Sn":0,
"Sb":0,"Te":0,"I":0,"Xe":0,"Cs":0,"Ba":0,"La":0,"Ce":0,"Pr":0,"Nd":0,
"Pm":0,"Sm":0,"Eu":0,"Gd":0,"Tb":0,"Dy":0,"Ho":0,"Er":0,"Tm":0,"Yb":0,
"Lu":0,"Hf":0,"Ta":0,"W":0,"Re":0,"Os":0,"Ir":0,"Pt":0,"Au":0,"Hg":0,
"Tl":0,"Pb":0,"Bi":0,"Po":0,"At":0,"Rn":0,"Fr":0,"Ra":0,"Ac":0,"Th":0,
"Pa":0,"U":0,"Np":0,"Pu":0,"Am":0,"Cm":0,"Bk":0,"Cf":0,"Es":0,"Fm":0,
"Md":0,"No":0,"Lr":0,"Rf":0,"Db":0,"Sg":0,"Bh":0,"Hs":0,"Mt":0}

#fragments = df["Molecular Formula"].apply(lambda x: pd.Series(x.split(",")))
#print fragments

molform = df['Molecular Formula'].to_string()
print molform
j = 0
for elements in molform:
    split = re.split("([a-z]+)([0-9]+)", molform, re.I)
    j+=1
    if j == 10:
        break
print split

In [ ]:
fragments = df["Molecular Formula"].str.split(",").apply(pd.Series)
fragments.columns = ('Fragment 1','Fragment 2','Fragment 3')
pd.concat([df2,fragments], axis=1)


For unknown reasons the concatination doesn't show when df is printed, but fragments show when printed solo.

In [ ]:
molform = pd.Series(df["Molecular Formula"])
for elements in molform:
    elementSymbol = molform.str.split('[A-Z]')
    elementNumber = molform.str.split('[0-9]') #<< this version prints all the numbers and symbols but grouped together


print elementSymbol
print elementnumber


df['elementSymbol'] = df['Molecular Formula'].str.extract('([A-Z])', expand=True) #<<these
df['elementNumber'] = df['Molecular Formula'].str.extract('(\d)', expand=True) #<<two don't extract in the form I want, result shown below

j = 0
with open('test2.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        molrow = row[2]
        molparsed = re.findall(r'([A-Z][a-z]*)(\d*)', molrow)
        j+=1
        if j==10:
            break
        print molparsed #works, was initially indented incorrectly.


after some searching around this is the closest I've come yet, although they are seperated the integers are grouped together, as are the characters, need to find a way to make sure the symbol is grouped with its correspondent value.
initially:
<pre>
df['elementSymbol'] = df['Molecular Formula'].str.extract('([A-Z])', expand=True)
df['elementNumber'] = df['Molecular Formula'].str.extract('(\d)', expand=True)
</pre>
was used however resulted in an incorrect output as such: 
```
   |  Z Prime  |  elementSymbol| elementNumber |  
---|-----------|---------------|---------------|               
   |           |         C     |        1      |
1  |   1.00    |         C     |        1      |
2  |   1.00    |         C     |        1      |
4  |   0.50    |         C     |        2      |
5  |   1.00    |         C     |        5      |
6  |   1.00    |         C     |        2      | 
7  |   0.25    |         C     |        8      | 
8  |   1.00    |         C     |        1      |
9  |   1.00    |         C     |        8      |
```
therefore was replaced with 
<pre>

molform = pd.Series(df["Molecular Formula"])
for elements in molform:
    elementSymbol = molform.str.split('[A-Z]')
    elementNumber = molform.str.split('[0-9]')

</pre>
which works better but still not what is neccessary.

The final attempt where the csv file is read and parsed works well but needs to be adapted to not use a counter and file.

In [ ]:
def molparsed(row):
    baseformula = {"H":0,"He":0,"Li":0,"Be":0,"B":0,"C":0,"N":0,"O":0,"F":0,"Ne":0,
    "Na":0,"Mg":0,"Al":0,"Si":0,"P":0,"S":0,"Cl":0,"Ar":0,"K":0,"Ca":0,
    "Sc":0,"Ti":0,"V":0,"Cr":0,"Mn":0,"Fe":0,"Co":0,"Ni":0,"Cu":0,"Zn":0,
    "Ga":0,"Ge":0,"As":0,"Se":0,"Br":0,"Kr":0,"Rb":0,"Sr":0,"Y":0,"Zr":0,
    "Nb":0,"Mo":0,"Tc":0,"Ru":0,"Rh":0,"Pd":0,"Ag":0,"Cd":0,"In":0,"Sn":0,
    "Sb":0,"Te":0,"I":0,"Xe":0,"Cs":0,"Ba":0,"La":0,"Ce":0,"Pr":0,"Nd":0,
    "Pm":0,"Sm":0,"Eu":0,"Gd":0,"Tb":0,"Dy":0,"Ho":0,"Er":0,"Tm":0,"Yb":0,
    "Lu":0,"Hf":0,"Ta":0,"W":0,"Re":0,"Os":0,"Ir":0,"Pt":0,"Au":0,"Hg":0,
    "Tl":0,"Pb":0,"Bi":0,"Po":0,"At":0,"Rn":0,"Fr":0,"Ra":0,"Ac":0,"Th":0,
    "Pa":0,"U":0,"Np":0,"Pu":0,"Am":0,"Cm":0,"Bk":0,"Cf":0,"Es":0,"Fm":0,
    "Md":0,"No":0,"Lr":0,"Rf":0,"Db":0,"Sg":0,"Bh":0,"Hs":0,"Mt":0}
    formula = row["Molecular Formula"]
    Z = row["Z"]
    parsedform = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    for element in parsedform:
        baseformula[element[0]] = float(element[1])*Z
        #print element
    return baseformula

A function to multiply the element number by the Z value to give a scaled version for the unit formula. For example C2H6O2 where 2 formula units in a unit cell = C4H12O4.

Below the function is applied to the dataframe.

In [ ]:
print df.apply(molparsed,axis=1)

In [ ]:
print df